### SatelliteBench

* [Dataset](https://physionet.org/content/multimodal-satellite-data/1.0.0/)

* [Paper](https://arxiv.org/abs/2401.11114)

In [ ]:
%cd ..

### Setup Environment:

In [2]:
import os
import pandas as pd

from src.classifiers import preprocess_data, process_labels,split_data

from src.classifiers import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_cpu_metrics import calculate_memory

In [3]:
PATH = 'Embeddings_vlm/satellitedata/'
FILE = 'embeddings_clip.csv'

FILE_PATH = os.path.join(PATH, FILE)

## Get data

In [4]:
df = pd.read_csv(FILE_PATH)
df.drop(columns=['text', 'image_id'], inplace=True)
df.Labels = df.Labels.astype(str)
df.head()

,Labels,split,image_embedding_0,image_embedding_1,image_embedding_2,image_embedding_3,image_embedding_4,image_embedding_5,image_embedding_6,image_embedding_7,...,text_embedding_504,text_embedding_505,text_embedding_506,text_embedding_507,text_embedding_508,text_embedding_509,text_embedding_510,text_embedding_511,Labels_4,Labels_3
0,1,train,-0.000648,0.047631,-0.007688,-0.015954,0.033954,-0.024987,0.008623,0.077579,...,0.014448,0.025682,0.010953,0.002404,0.021032,0.009018,0.002254,-0.014104,3,2
1,1,train,0.017359,0.030042,0.016325,-0.010410,0.046289,-0.025459,-0.002872,0.065161,...,0.012622,0.030623,0.007002,0.006732,0.010945,-0.010793,-0.000270,-0.012476,1,1
2,0,train,-0.022790,0.021263,0.040344,-0.016606,0.041269,-0.028196,0.012908,0.048379,...,-0.014684,0.023051,0.005047,0.000785,0.005979,-0.020611,-0.006694,0.008903,1,0
3,0,train,-0.013127,-0.022996,-0.049374,-0.006306,0.013601,-0.003762,-0.033799,0.111063,...,-0.011703,0.025439,0.004510,-0.007244,0.005063,0.042533,-0.002328,-0.006062,1,1
4,0,train,-0.034772,0.040687,0.007086,-0.026446,0.017892,-0.021177,0.013868,0.056864,...,0.022611,0.020556,0.020584,-0.001789,0.015643,-0.005070,0.008810,0.005491,1,0


## Data Perparation

In [5]:
# Split the data
train_df, test_df = split_data(df)

# Select features and labels vectors
text_columns = [column for column in df.columns if 'text' in column] #[f'text_{i}']
image_columns = [column for column in df.columns if 'image' in column] #[f'image_{i}']
label_columns = 'Labels'


# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)


train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Train Shape: (936, 1028)
Test Shape: (312, 1028)


### Models

In [6]:
text_input_size = len(text_columns)
image_input_size = len(image_columns)
if label_columns == 'DR_2':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [7]:
calculate_memory(train_loader, test_loader, text_input_size, image_input_size, output_size)

Early fusion:
Average Memory per Batch in Train: 0.24 MB
Total Memory Usage per Epoch Train: 3.66 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 0.08 MB
Total Memory Usage per Epoch Test: 0.41 MB (excluding model parameters)
Model: 
Model Memory Usage: 0.50 MB

Late fusion:
Average Memory per Batch in Train: 0.24 MB
Total Memory Usage per Epoch Train: 3.66 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 0.08 MB
Total Memory Usage per Epoch Test: 0.41 MB (excluding model parameters)
Model: 
Model Memory Usage: 0.25 MB
